In [0]:
# replaced the key with a placeholder for privacy
spark.conf.set(
"fs.azure.account.key.goodreadsreviews60104758.dfs.core.windows.net",
"<access-key>"
)

In [0]:
# load the data
gold_path_v3 = "abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/Gold_layer/features_v1"
df = spark.read.format("delta").load(gold_path_v3)
df.show(5)


+--------+-------------------+--------------------+--------------------+---------+--------------------+------+--------------------+-------------+-------+--------------+---------------+--------------------+-----------------+------------------+------------------+------------------+
| book_id|               name|           review_id|               title|author_id|             user_id|rating|         review_text|language_code|n_votes|date_added_iso|avg_book_rating|num_reviews_per_book|author_avg_rating|min_words_per_book|max_words_per_book|avg_words_per_book|
+--------+-------------------+--------------------+--------------------+---------+--------------------+------+--------------------+-------------+-------+--------------+---------------+--------------------+-----------------+------------------+------------------+------------------+
|17327982|        Vas Kolovos|78f5b377cf36b5230...|    Skin (flesh, #2)| 14186488|ebfdd03f8f0a13497...|     3|starts slow, but ...|      Unknown|      0|    

In [0]:
#Splitting the dataset
# Set random seed for reproducibility
RANDOM_SEED = 42

# Split 70% train, 30% temp
train_df, temp_df = df.randomSplit([0.7, 0.3], seed=RANDOM_SEED)

# 30% temp into 15% val and 15% test
val_df, test_df = temp_df.randomSplit([0.5, 0.5], seed=RANDOM_SEED)


In [0]:
# Checking counts
total_rows = df.count()
train_count = train_df.count()
val_count = val_df.count()
test_count = test_df.count()
total_check = train_count + val_count + test_count

# summary
print(f"{'Train':<15} {train_count:>15,} {train_count/total_rows*100:>14.2f}%")
print(f"{'Validation':<15} {val_count:>15,} {val_count/total_rows*100:>14.2f}%")
print(f"{'Test':<15} {test_count:>15,} {test_count/total_rows*100:>14.2f}%")
print(f"{'-'*70}")
print(f"{'Total':<15} {total_check:>15,} {100.00:>14.2f}%")
print(f"{'='*70}\n")

# Verify total
if total_check == total_rows:
    print("Split successful! Counts match original dataset.")
else:
    print(f"Warning: Totals do not match ({total_check:,} vs {total_rows:,}).")

Train                10,480,888          70.01%
Validation            2,245,513          15.00%
Test                  2,243,301          14.99%
----------------------------------------------------------------------
Total                14,969,702         100.00%

Split successful! Counts match original dataset.


In [0]:
# Save splits to gold layer
train_df.write.format("delta").mode("overwrite").save("abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/Gold_layer/features_v2/train")
val_df.write.format("delta").mode("overwrite").save("abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/Gold_layer/features_v2/val")
test_df.write.format("delta").mode("overwrite").save("abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/Gold_layer/features_v2/test")